In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv("input/train_indessa.csv")
en_train = pd.read_csv("ensemble_train.csv")
en_test = pd.read_csv("ensemble_test.csv")

In [16]:
data.shape

(532428, 45)

## Split Training Set

In [24]:
# One Hot Y
from sklearn.preprocessing import OneHotEncoder

data = data.fillna("0")
data = data.dropna()
split = int(len(data)*0.75)

oht_target = OneHotEncoder()
Y = pd.DataFrame(oht_target.fit_transform(data[data['loan_status'] >= 0].iloc[:,-1:]).todense())
Y_train = Y[:split]
Y_test = Y[split:]

In [25]:
print(Y.shape)
print(Y_train.shape)
print(Y_test.shape)
print(split)

(532428, 2)
(399321, 2)
(133107, 2)
399321


## Ensemble with ANN

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.advanced_activations import PReLU

Using TensorFlow backend.


In [26]:
model = Sequential()

In [27]:
model.add(Dense(units=40, input_dim=9))
model.add(Activation('relu'))
model.add(Dense(units=40))
model.add(Activation('relu'))
model.add(Dense(units=40))
model.add(Activation('relu'))
model.add(Dense(units=40))
model.add(Activation('relu'))
model.add(Dense(units=40))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [28]:
model.fit(en_train.values, Y_train.values, epochs=10, batch_size=32)

Epoch 1/10
399321/399321 [==============================] - 20s - loss: 0.0012 - acc: 0.9999        

In [30]:
loss_metrics = model.evaluate(en_test.values, Y_test.values, batch_size=128)

128640/133107 [===========================>..] - ETA: 0s 

In [31]:
loss_metrics

[1.4448285770793123, 0.89225209792012616]

## Prediction

In [39]:
# Get member id
data_test = pd.read_csv("input/test_indessa.csv")
rows = data_test['member_id'].copy()

In [58]:
# Predict Ensemble with ANN
en_new = pd.read_csv("ensemble_new.csv")

In [38]:
pred_test = model.predict_proba(en_new.values)

352352/354951 [============================>.] - ETA: 0s  

In [61]:
# Bagging Prob
pred_bc = pd.read_csv("submission_bagging.csv")

In [62]:
# Gradient Boosting Prob
pred_gbc = pd.read_csv("submission_gbc.csv")

In [63]:
pred_frame = pd.DataFrame({'loan_status': pred_test[:,1],
                           'gbc': pred_bc['loan_status'],
                           'bc': pred_gbc['loan_status']})

In [64]:
def rep(x):
    if((x['loan_status'] == 0) or (x['loan_status'] == 1)):
        return (x['gbc']+x['bc'])/2
    else:
        return (x['gbc']+x['bc']+x['loan_status'])/3
pred_frame_mean = pred_frame.apply(rep, axis=1)

In [66]:
pred_frame_mean = pred_frame_mean.round(2)

In [67]:
pd.DataFrame({
    'member_id': rows,
    'loan_status': pred_frame_mean
}).to_csv('submission_final.csv', index=False, columns=['member_id', 'loan_status'], float_format='%g')